In [5]:
def convert_frame(datetime):
    minute = int(datetime.split(':')[0])
    second = int(datetime.split(':')[1].split('.')[0])
    frame = int(datetime.split(':')[1].split('.')[1])

    total_frame = frame + 60 * second + 3600 * minute

    return total_frame

In [6]:
dic = {
    "label": "M3",
    "start": "00:00.00",
    "end": "00:00.21",
    "duration": "00:00.21"
  }

start_frame = convert_frame(dic["start"]) + 1
end_frame = convert_frame(dic["end"])

print(start_frame)
print(end_frame)

1
21


In [4]:
import os
import json

RESULT = './results'
project_id = 1

with open(os.path.join(RESULT, 'temp.json'), 'r') as f:
    data = json.load(f)
for dic in data:
    dic['topk'] = [{'label': 'M3', 'score':0.94}]
    
with open(os.path.join(RESULT, 'temp.json'), 'w') as f:
    json.dump(data, f)
